# yield

In [3]:
import inspect

def looper():
    print("started")
    for i in range(3):
        yield i

loop = looper()
print(inspect.getgeneratorstate(loop))
print(next(loop))
print(inspect.getgeneratorstate(loop))

GEN_CREATED
started
0
GEN_SUSPENDED


In [9]:
# 通过生成器表达式
loop = (x*x for x in range(3))
print(inspect.getgeneratorstate(loop))
print(next(loop))
print(inspect.getgeneratorstate(loop))

GEN_CREATED
0
GEN_SUSPENDED


In [4]:
def adder():
    ret = 0
    while True:
        i = yield ret
        ret += i

looper = adder()
next(looper)

for n in range(1, 10):
    i = looper.send(n)
    print(i)

looper.close()
print(inspect.getgeneratorstate(looper))

1
3
6
10
15
21
28
36
45
GEN_CLOSED


In [5]:
def adder():
    ret = 0
    while True:
        i = yield ret
        ret += i


looper = adder()
next(looper)

for n in range(1, 10):
    looper.send(n)

looper.close()
print(inspect.getgeneratorstate(looper))  # GEN_CLOSED

# next(looper) 
# raise StopIteration

GEN_CLOSED


In [6]:
def echo():
    i = 0
    while True:
        i += 1
        try:
            yield i
            print('aha')
        except TypeError as ex:
            print(ex, i)
        except GeneratorExit:
            print("gen exit", i)
            return


g = echo()
print("from generator:", next(g))
print(g.throw(TypeError, "TypeError: from caller"))

print(next(g))
print(next(g))
print(next(g))

print("END")

from generator: 1
TypeError: from caller 1
2
aha
3
aha
4
aha
5
END


In [7]:
def adder():
    ''' 子生成器
    '''
    ret = 0
    while True:
        n = yield
        if n is None:
            break
        ret += n
    return ret


def worker(result, key):
    ''' 委派生成器
    '''
    while True:
        rv = yield from adder()
        result[key] = rv


def main():
    ''' 调用方
    '''
    result = {}
    for i in range(1, 3):
        w = worker(result, i)
        next(w)
        for j in range(i, i+3):
            w.send(j)
        w.send(None)
        print(inspect.getgeneratorstate(w))
    print(result)


main()


GEN_SUSPENDED
GEN_SUSPENDED
{1: 6, 2: 9}
